In [ ]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings
from bs4 import BeautifulSoup
import re
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
#Removing negations
j=0
for i in data['tweet']:
    tweet=i
    CONTRACTIONS = {"mayn't":"may not", "may've":"may have", "isn't":"is not", "they're":"they are", "wasn't":"was not", "weren't":"were not", "doesn't":"does not", "don't":"do not", "didn't":"did not", "hasn't":"has not", "we've":"we have", "hadn't":"had not", "can't":"can not", "couldn't":"could not", "shouldn't":"should not", "mustn't":"must not","won't":"wont" , "might've":"might have"}
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    data['tweet'][j] = " ".join(reformed)
    j=j+1
for col in data['tweet']:
    print(col)

In [ ]:
data.shape

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [ ]:
# remove twitter handles (@user)
data['tweet'] = np.vectorize(remove_pattern)(data['tweet'], "@[\w]*")

In [ ]:
#Removing url
def remove_url(text):
    return re.sub(r"http\S+", "", text)

In [ ]:
data['tweet']=data['tweet'].apply(remove_url)

In [ ]:
# remove special characters, numbers, punctuations
data['tweet'] = data['tweet'].str.replace("[^a-zA-Z]", " ")

In [ ]:
#Converting to lowercase
j=0
for i in data['tweet']:
    data['tweet'][j]=data['tweet'][j].lower()
    j=j+1
for col in data['tweet']: 
    print(col)

In [ ]:
#Replace elongated words
def replaceElongated(word):
    """ Replaces an elongated word with its basic form, unless the word exists in the lexicon """

    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if wordnet.synsets(word):
        return word
    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:      
        return replaceElongated(repl_word)
    else:       
        return repl_word

In [ ]:
def removeEmoticons(text):
    """ Removes emoticons from text """
    text = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', text)
    return text

In [ ]:
data['tweet']=data['tweet'].apply(removeEmoticons)

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [ ]:
#Apply function on review column
data['tweet']=data['tweet'].apply(remove_stopwords)
for col in data['tweet']: 
    print(col)

In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [ ]:
data['tweet']=data['tweet'].apply(denoise_text)

In [ ]:
#Removing Short Words
data['tweet'] = data['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))


In [ ]:
#Lemmatization
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
def lemmat(text):
    stopword = stopwords.words('english')
    wordnet_lemmatizer = WordNetLemmatizer()
    word_tokens = nltk.word_tokenize(text)
    text =' '.join([wordnet_lemmatizer.lemmatize(word) for word in word_tokens])
    return text

#Stemming
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text=' '.join([ps.stem(word) for word in text.split()])
    return text


In [ ]:
data['tweet'] = data['tweet'].apply(simple_stemmer)
data['tweet'] = data['tweet'].apply(lemmat)
for col in data['tweet']: 
    print(col)

In [ ]:
#Splitting data for testing and training
x=data['tweet']
y = data['label']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.3)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import FreqDist
all_words=' '.join(x_train)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
dist

In [ ]:
num_unique_word=len(dist)
num_unique_word

import numpy as np
r_len=[]
for text in x_train:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

In [ ]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN

from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))

In [ ]:
#Texts to sequences
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)
x_train

In [ ]:
X_train = np.asarray(x_train)
X_test = np.asarray(x_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
X_train

In [ ]:
from keras.preprocessing import sequence
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_test.shape)
X_train

In [ ]:
#Importing models
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Embedding,SimpleRNN,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional

### Model

In [ ]:
#Model building - Conv1D
from tensorflow.python.keras.layers import Dropout,Conv1D, GlobalMaxPooling1D

model= Sequential()

model.add(Embedding(max_features, 50,input_length=38))
model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model.add(Dropout(0.2))

model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
history=model.fit(X_train, y_train,epochs=4, validation_split=0.2)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
#Model2
model2 = Sequential()

model2.add(Embedding(max_features, 100, input_length=38))
model2.add(SpatialDropout1D(0.25))
model2.add(Bidirectional(GRU(128)))
model2.add(Dropout(0.5))

model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

In [ ]:
history2=model2.fit(X_train, y_train,epochs=5, validation_split=0.2, batch_size=40)

In [ ]:
model2.evaluate(X_test,y_test)